In [1]:
pip install pymysql

In [2]:
pip install faker

Note: you may need to restart the kernel to use updated packages.


In [1]:
from faker import Faker
import random
import pymysql.cursors
from datetime import datetime, timedelta
import bcrypt

# Connect to the database
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='',
    db='e_commerce',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

fake = Faker()

def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

def hash_account_number(account_number):
    return bcrypt.hashpw(account_number.encode('utf-8'), bcrypt.gensalt())

fake = Faker()

def generate_fake_users(num_users):
    users = []
    for _ in range(num_users):
        user = {
            'Username': fake.user_name(),
            'Password_user': fake.password(),
            'FirstName': fake.first_name(),
            'LastName': fake.last_name(),
            'Email': fake.email(),
            'PhoneNumber': fake.phone_number(),
            'DateOfBirth': fake.date_of_birth(minimum_age=18, maximum_age=65).strftime('%Y-%m-%d')
        }
        users.append(user)
    return users

def generate_fake_addresses(num_addresses, user_ids):
    addresses = []
    for _ in range(num_addresses):
        address = {
            'UserID': random.choice(user_ids),
            'StreetAddress': fake.street_address(),
            'City': fake.city(),
            'State': fake.state(),
            'PostalCode': fake.zipcode(),
            'Country': fake.country()
        }
        addresses.append(address)
    return addresses

def generate_fake_products(num_products):
    products = []
    for _ in range(num_products):
        product = {
            'ProductName': fake.word(),
            'Description': fake.text(),
            'Price': round(random.uniform(10.0, 100.0), 2),
            'StockQuantity': random.randint(10, 100),
            'Manufacturer': fake.company(),
            'Category': fake.word()
        }
        products.append(product)
    return products

def generate_fake_carts(num_carts, user_ids, product_ids):
    carts = []
    for _ in range(num_carts):
        cart = {
            'UserID': random.choice(user_ids),
            'ProductID': random.choice(product_ids),
            'Quantity': random.randint(1, 5),
            'AddedToCartDate': fake.date_time_this_year(before_now=True, after_now=False, tzinfo=None).strftime('%Y-%m-%d %H:%M:%S')
        }
        carts.append(cart)
    return carts

def generate_fake_commands(num_commands, user_ids, product_ids):
    commands = []
    for _ in range(num_commands):
        command = {
            'UserID': random.choice(user_ids),
            'ProductID': random.choice(product_ids),
            'Quantity': random.randint(1, 5),
            'OrderDate': fake.date_time_this_year(before_now=True, after_now=False, tzinfo=None).strftime('%Y-%m-%d %H:%M:%S'),
            'Status': random.choice(['Processing', 'Shipped', 'Delivered'])
        }
        commands.append(command)
    return commands

def generate_fake_invoices(num_invoices, user_ids, command_ids, payment_ids):
    invoices = []
    for _ in range(num_invoices):
        invoice = {
            'UserID': random.choice(user_ids),
            'CommandID': random.choice(command_ids),
            'TotalAmount': round(random.uniform(50.0, 500.0), 2),
            'InvoiceDate': fake.date_time_this_year(before_now=True, after_now=False, tzinfo=None).strftime('%Y-%m-%d %H:%M:%S'),
            'PaymentMethodID': random.choice(payment_ids)
        }
        invoices.append(invoice)
    return invoices

def generate_fake_rates(num_rates, user_ids, product_ids):
    rates = []
    for _ in range(num_rates):
        rate = {
            'ProductID': random.choice(product_ids),
            'UserID': random.choice(user_ids),
            'Rating': random.randint(1, 5),
            'Review': fake.text()
        }
        rates.append(rate)
    return rates

def generate_fake_payments(num_payments, user_ids):
    payments = []
    for _ in range(num_payments):
        payment = {
            'UserID': random.choice(user_ids),
            'PaymentType': random.choice(['Credit Card', 'PayPal', 'Bank Transfer']),
            'AccountNumber': fake.credit_card_number()
        }
        payments.append(payment)
    return payments


try:
    with connection.cursor() as cursor:
        # Generate and insert fake data into User_table
        fake_users = generate_fake_users(10)
        for user in fake_users:
            hashed_password = hash_password(user['Password_user'])
            cursor.execute("INSERT INTO User_table (Username, Password_user, FirstName, LastName, Email, PhoneNumber, DateOfBirth) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                           (user['Username'], hashed_password, user['FirstName'], user['LastName'], user['Email'], user['PhoneNumber'], user['DateOfBirth']))
        connection.commit()

        # Get generated user IDs
        cursor.execute("SELECT UserID FROM User_table")
        user_ids = [user['UserID'] for user in cursor.fetchall()]

        # Generate and insert fake data into Address
        fake_addresses = generate_fake_addresses(10, user_ids)
        for address in fake_addresses:
            cursor.execute("INSERT INTO Address (UserID, StreetAddress, City, State, PostalCode, Country) VALUES (%s, %s, %s, %s, %s, %s)",
                           (address['UserID'], address['StreetAddress'], address['City'], address['State'], address['PostalCode'], address['Country']))
        connection.commit()

        # Generate and insert fake data into Product
        fake_products = generate_fake_products(10)
        for product in fake_products:
            cursor.execute("INSERT INTO Product (ProductName, Description, Price, StockQuantity, Manufacturer, Category) VALUES (%s, %s, %s, %s, %s, %s)",
                           (product['ProductName'], product['Description'], product['Price'], product['StockQuantity'], product['Manufacturer'], product['Category']))
        connection.commit()

        # Get generated product IDs
        cursor.execute("SELECT ProductID FROM Product")
        product_ids = [product['ProductID'] for product in cursor.fetchall()]

        # Generate and insert fake data into Cart
        fake_carts = generate_fake_carts(10, user_ids, product_ids)
        for cart in fake_carts:
            cursor.execute("INSERT INTO Cart (UserID, ProductID, Quantity, AddedToCartDate) VALUES (%s, %s, %s, %s)",
                           (cart['UserID'], cart['ProductID'], cart['Quantity'], cart['AddedToCartDate']))
        connection.commit()

        # Generate and insert fake data into Command
        fake_commands = generate_fake_commands(10, user_ids, product_ids)
        for command in fake_commands:
            cursor.execute("INSERT INTO Command (UserID, ProductID, Quantity, OrderDate, Status) VALUES (%s, %s, %s, %s, %s)",
                           (command['UserID'], command['ProductID'], command['Quantity'], command['OrderDate'], command['Status']))
        connection.commit()

        # Get generated command IDs
        cursor.execute("SELECT CommandID FROM Command")
        command_ids = [command['CommandID'] for command in cursor.fetchall()]

        # Generate and insert fake data into Invoices
        fake_invoices = generate_fake_invoices(10, user_ids, command_ids, product_ids)
        for invoice in fake_invoices:
            cursor.execute("INSERT INTO Invoices (UserID, CommandID, TotalAmount, InvoiceDate, PaymentMethodID) VALUES (%s, %s, %s, %s, %s)",
                           (invoice['UserID'], invoice['CommandID'], invoice['TotalAmount'], invoice['InvoiceDate'], invoice['PaymentMethodID']))
        connection.commit()

        # Get generated payment IDs
        cursor.execute("SELECT PaymentID FROM Payment")
        payment_ids = [payment['PaymentID'] for payment in cursor.fetchall()]

        # Generate and insert fake data into Rate
        fake_rates = generate_fake_rates(10, user_ids, product_ids)
        for rate in fake_rates:
            cursor.execute("INSERT INTO Rate (ProductID, UserID, Rating, Review) VALUES (%s, %s, %s, %s)",
                           (rate['ProductID'], rate['UserID'], rate['Rating'], rate['Review']))
        connection.commit()

        # Generate and insert fake data into Payment
        fake_payments = generate_fake_payments(10, user_ids)
        for payment in fake_payments:
            hashed_account_number = hash_account_number(payment['AccountNumber'])
            cursor.execute("INSERT INTO Payment (UserID, PaymentType, AccountNumber) VALUES (%s, %s, %s)",
                           (payment['UserID'], payment['PaymentType'], hashed_account_number))
        connection.commit()

finally:
    connection.close()



In [ ]:
https://github.com/yassinefri/Faker.git